# Modified Ochiai 2.1
Fake failing tests set: All the tests covering the higher number possible of methods in the stack traces (in other words, cover more of these methods)
The number of lines covered is not taken into consideration, if it covers at least 1 line of the method, than it covers the method
Obs: the difference when compared to Ochiai 2.0 is that this script does not use the real failing tests

In [4]:
import os
from my_secrets import base_path

paths_dict=  {
    "gzoltar_files_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "gzoltar_files"),
    "output_file": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiScores", "modifiedOchiai2.1"),
    "data_file_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
    "failing_tests_info_file_name": "modifiedOchiai2.1_fake_failing_tests_info",
    "tests_analysis_results":  os.path.join(base_path,"DeepDiveBugReportsWithLogs",  "data", "rq1_results.json"),
    "fake_test_results_file_name": "fake_test_results_modifiedOchiai2.1.csv",
    "tests_covering_stack_traces_folder":  os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "tests_covering_stack_traces_details_per_bug")
}

## Reading the coverage data

In [5]:
%run ../utils.py

bugs_data = json_file_to_dict(paths_dict["data_file_path"])
tests_analysis_results  = json_file_to_dict(paths_dict["tests_analysis_results"])
bugs = get_list_of_bugs_with_coverage(tests_analysis_results)
coverage_data = {}

for bug in bugs:
    project, bug_id = bug.split("_")
    project_gzoltar_folder = os.path.join(paths_dict["gzoltar_files_path"], project)
    if not os.path.exists(project_gzoltar_folder):
        print("Gzoltar folder not fount for the project "+ project)
        print("Skipping!!!!! ")
        continue
    if not project in coverage_data.keys():
        coverage_data[project] = {}

    bug_gzoltar_folder = os.path.join(project_gzoltar_folder, bug_id)
    if not os.path.exists(bug_gzoltar_folder):
        print("Gzoltar folder not fount for the bugId "+ project+ "-" + bug_id)
        print("Skipping!!!!! ")
        continue
    coverage = {}
    try:
        coverage["methods_covered_per_test"] = read_methods_matrix_file(bug_gzoltar_folder)
        coverage["methods_obj_list"] = read_methods_spectra_file(bug_gzoltar_folder)
        test_names, test_results  = read_tests_csv_to_lists(bug_gzoltar_folder)
        coverage["test_names"] = test_names
        print("Number of tests in bug " + project + "-" + bug_id + " - " + str(len(test_names)))
        coverage["test_results"] = [True] * len(test_names) # Erasing the real test results
        coverage_data[project][bug_id] = coverage
    except FileNotFoundError:
        print ("The bug " + project + "-" + bug_id + " does not contain one the the required files. Skipping it")
        continue

print("Done")

Number of tests in bug Cli-14 - 539
Number of tests in bug Cli-5 - 103
Number of tests in bug Closure-106 - 2554
Number of tests in bug Closure-125 - 8156
Number of tests in bug Closure-143 - 4616
Number of tests in bug Closure-152 - 6186
Number of tests in bug Closure-2 - 7830
Number of tests in bug Closure-34 - 7413
Number of tests in bug Closure-37 - 7373
Number of tests in bug Closure-55 - 7155
Number of tests in bug Codec-8 - 294
Number of tests in bug Collections-28 - 5280
Number of tests in bug Compress-1 - 69
Number of tests in bug Compress-12 - 315
Number of tests in bug Compress-14 - 317
Number of tests in bug Compress-17 - 350
Number of tests in bug Compress-18 - 351
Number of tests in bug Compress-23 - 466
Number of tests in bug Compress-24 - 935
Number of tests in bug Compress-27 - 1012
Number of tests in bug Compress-31 - 1106
Number of tests in bug Compress-32 - 613
Number of tests in bug Compress-34 - 1228
Number of tests in bug Csv-12 - 206
Number of tests in bug Csv-4

## Defining the fake failing tests and Running Ochiai

In [6]:
%run ../utils.py
import math

fake_failing_tests_info = {}
for project in coverage_data.keys():
    for bug_id in coverage_data[project].keys():

        bug_data = bugs_data[project][bug_id]
        coverage = coverage_data[project][bug_id]

        print(project + " ---- " + bug_id)

        buggy_commit = bug_data["buggy_commit"]
        fake_failed_tests = []
        methods_list = []

        if bug_data["stackTraceMethodsDetails"] =={ }:
            print("The bug does not contain stackTraceMethodsDetails. Skipping.")
            print()
            continue

        print("* Part 1 - defining the fake failing tests")
        tests_covering_stack_traces_file_path = os.path.join(paths_dict["tests_covering_stack_traces_folder"], project, bug_id + ".json")
        tests_covering_stack_traces_details = json_file_to_dict(tests_covering_stack_traces_file_path)
        stack_traces_test_count = {}
        for st_file in bug_data["stackTraceMethodsDetails"].keys():
            for st_method in bug_data["stackTraceMethodsDetails"][st_file].keys():
                if "tests_covering_the_method" in tests_covering_stack_traces_details[st_file][st_method].keys():
                    for test in tests_covering_stack_traces_details[st_file][st_method]["tests_covering_the_method"]:
                        if test in stack_traces_test_count.keys():
                            stack_traces_test_count[test] += 1
                        else:
                            stack_traces_test_count[test] = 1
        if stack_traces_test_count!= {}:
            threshold = max(stack_traces_test_count.values())
        else:
            print("No failed tests found in the stack trace")
            threshold = 1
        print("Threshold: " + str(threshold))

        print("* Part 2 - preparing the fake_tests_status")
        coverage["fake_test_results"] = coverage["test_results"]
        for index, test in enumerate(coverage["test_names"]):
            if test in stack_traces_test_count.keys() and stack_traces_test_count[test] >= threshold:
                coverage["fake_test_results"][index] = False
            else:
                coverage["fake_test_results"][index] = True

        number_of_failing_tests = 0
        for test_result in coverage["fake_test_results"]:
            if not test_result:
                number_of_failing_tests += 1

        if number_of_failing_tests == 0:
            print(f"The bug {project}_{bug_id} does not contain fake failing tests. Skipping it")
            print()
            continue

        print("Storing the fake test results from this implementation in a file for future uses")
        store_fake_test_results(coverage, project, bug_id, paths_dict["gzoltar_files_path"], paths_dict["fake_test_results_file_name"])
        print("store_fake_test_results operation completed")

        print("* Part 4 - Executing Ochiai")
        methods_ochiai_scores = {}
        for index_m, method_name in enumerate(coverage["methods_obj_list"]):
            n00 = 0
            n01 = 0
            n10 = 0
            n11 = 0
            s_o = 0
            for index_t, test_name in enumerate(coverage["test_names"]):
                if str(coverage["methods_covered_per_test"][index_t][index_m]) == "1":
                    if not coverage["fake_test_results"][index_t]:
                        n11 += 1
                    else:
                        n10 += 1
                else:
                    if not coverage["fake_test_results"][index_t]:
                        n01 += 1
                    else:
                        n00 += 1
            try:
                s_o = n11/math.sqrt((n11+n01)*(n11+n10))
            except ZeroDivisionError:
                s_o = 0
            methods_ochiai_scores[method_name] = s_o

        if project not in fake_failing_tests_info.keys():
            fake_failing_tests_info[project] = {}

        fake_failing_tests_info[project][bug_id] = {}
        fake_failing_tests_info[project][bug_id]["fake_passing_tests_number"] = len(coverage["fake_test_results"]) - number_of_failing_tests
        fake_failing_tests_info[project][bug_id]["fake_failing_tests_number"] = number_of_failing_tests

        print("Number of fake passing tests: " + str(fake_failing_tests_info[project][bug_id]["fake_passing_tests_number"]))
        print("Number of fake failing tests: " + str(fake_failing_tests_info[project][bug_id]["fake_failing_tests_number"]) + "\n")
        dict_to_json_file(os.path.join(paths_dict["output_file"], project, bug_id), methods_ochiai_scores)

dict_to_json_file(os.path.join(paths_dict["output_file"], paths_dict["failing_tests_info_file_name"]), fake_failing_tests_info)
print("Execution completed")

Cli ---- 14
* Part 1 - defining the fake failing tests
Threshold: 6
* Part 2 - preparing the fake_tests_status
Storing the fake test results from this implementation in a file for future uses
store_fake_test_results operation completed
* Part 4 - Executing Ochiai
Number of fake passing tests: 538
Number of fake failing tests: 1

Cli ---- 5
* Part 1 - defining the fake failing tests
Threshold: 3
* Part 2 - preparing the fake_tests_status
Storing the fake test results from this implementation in a file for future uses
store_fake_test_results operation completed
* Part 4 - Executing Ochiai
Number of fake passing tests: 43
Number of fake failing tests: 60

Cli ---- 7
The bug does not contain stackTraceMethodsDetails. Skipping.

Closure ---- 106
The bug does not contain stackTraceMethodsDetails. Skipping.

Closure ---- 125
* Part 1 - defining the fake failing tests
Threshold: 14
* Part 2 - preparing the fake_tests_status
Storing the fake test results from this implementation in a file for f